![Logo](https://docs.openmc.org/en/latest/_static/openmc_logo.png)

# Importing libraries

In [ ]:
import sys
import openmc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [ ]:
sys.path.append('/home/ssartur/Documents/OpenMC/NuReMa')
from thermophysicaldata import *

# Building Angra 1 core

## Inputs

Pitch obtained from Luciano's thesis. \
Hot zero power data obtained from Westinghouse's Angra 1 Cycle 1 Data Book.

In [ ]:
# Fuel rod pitch:

pitch = 1.233598

# Global temperature:

T = 564.82 # K

# Fuel temperature:

Tf = 564.82 # K

# Coolant pressure:

p = 15.51 # MP

# Helium pressure:

p_He = 3e6 # P

# Boron concentration:

boron_ppm = 0 #ppm

## Materials

Materials compositons and densities obtained from BEAVRS. \
Uranium enrichments obtained from Westinghouse's Angra 1 Cycle 1 Data Book. \
Materials densities as functions of temperature and/or pressure obtained from IAEA's Thermophysical Properties of Materials for Nuclear Engineering.

In [ ]:
# Uranium dioxide 2.1% enriched:

UO2_21 = openmc.Material(1, 'UO2 2.1%', temperature = Tf)
UO2_21.add_element('U', 1.0, enrichment = 2.1)
UO2_21.add_nuclide('O16', 2.0)
UO2_21.set_density('g/cm3', UO2_density(Tf))
print(UO2_21)

# Uranium dioxide 2.6% enriched:

UO2_26 = openmc.Material(2, 'UO2 2.6%', temperature = Tf)
UO2_26.add_element('U', 1.0, enrichment = 2.6)
UO2_26.add_nuclide('O16', 2.0)
UO2_26.set_density('g/cm3', UO2_density(Tf))
print(UO2_26)

# Uranium dioxide 3.1% enriched:

UO2_31 = openmc.Material(3, 'UO2 3.1%', temperature = Tf)
UO2_31.add_element('U', 1.0, enrichment = 3.1)
UO2_31.add_nuclide('O16', 2.0)
UO2_31.set_density('g/cm3', UO2_density(Tf))
print(UO2_31)

# Helium:

Helium = openmc.Material(4, 'Helium', temperature = T)
Helium.add_nuclide('He4', 1.0)
Helium.set_density('g/cm3', helium_density(T, p_He))
print(Helium)

# Zircaloy 4:

Zircaloy = openmc.Material(5, 'Zircaloy 4', temperature = T)
Zircaloy.add_element('Zr', 0.98115, 'wo')
Zircaloy.add_element('Cr', 0.0010, 'wo')
Zircaloy.add_element('Sn', 0.0145, 'wo')
Zircaloy.add_element('Fe', 0.0021, 'wo')
Zircaloy.add_nuclide('O16', 0.00125, 'wo')
Zircaloy.set_density('g/cm3', zirconium_density(T))
print(Zircaloy)

# Borated water:
    
H2O = openmc.Material(6, 'Borated Water', temperature = T)
H2O.add_nuclide('H1', 2.0)
H2O.add_nuclide('O16', 1.0)
H2O.add_element('B', boron_ppm*(1e-6))
H2O.set_density('g/cm3', openmc.data.water_density(T, p))
H2O.add_s_alpha_beta('c_H_in_H2O')
print(H2O)

# Control rod absorber alloy:

AgInCd = openmc.Material(7, 'AgInCd')
AgInCd.add_element('Ag', 0.80, 'wo')
AgInCd.add_element('In', 0.15, 'wo')
AgInCd.add_element('Cd', 0.05, 'wo')
AgInCd.set_density('g/cm3', 10.16)
print(AgInCd)

# Borosilicate glass (burnable poison):

Borosilicate = openmc.Material(8, 'Borosilicate Glass')
Borosilicate.add_nuclide('O16', 0.5481, 'wo')
Borosilicate.add_element('Si', 0.3787, 'wo')
Borosilicate.add_element('Al', 0.0344, 'wo')
Borosilicate.add_nuclide('B10', 0.0071, 'wo')
Borosilicate.add_nuclide('B11', 0.0317, 'wo')
Borosilicate.set_density('g/cm3', 2.26)
print(Borosilicate)

# Stainless steel 304:

Steel = openmc.Material(9, 'Stainless Steel 304')
Steel.add_element('Fe', 0.6840, 'wo')
Steel.add_element('Ni', 0.1000, 'wo')
Steel.add_element('Cr', 0.1900, 'wo')
Steel.add_element('Mn', 0.0200, 'wo')
Steel.add_element('Si', 0.0060, 'wo')
Steel.set_density('g/cm3', 8.03)
print(Steel)

# Inconel 718:

Inconel = openmc.Material(10, 'Inconel 718')
Inconel.add_element('Fe', 0.2863, 'wo')
Inconel.add_element('Ni', 0.5119, 'wo')
Inconel.add_element('Cr', 0.1896, 'wo')
Inconel.add_element('Mn', 0.0087, 'wo')
Inconel.add_element('Si', 0.0035, 'wo')
Inconel.set_density('g/cm3', 8.2)
print(Inconel)

# Air:

Air = openmc.Material(11, 'Air')
Air.add_nuclide('O16', 0.2095 + 0.0048, 'wo')
Air.add_nuclide('N14', 0.7809 + 0.0048, 'wo')
Air.set_density('g/cm3', 0.000616)
print(Air)

# Exporting the materials list to XML and indicate the address to the nuclear data library:

materials = openmc.Materials([UO2_21, UO2_26, UO2_31, Helium, Zircaloy, H2O, AgInCd, Borosilicate, Steel, Inconel, Air])
materials.cross_sections = '/home/ssartur/Documents/Nuclear Data/ENDFB 8.0/cross_sections.xml'
materials.export_to_xml()

## Geometry

### Fuel rod geometry

Fuel rod radii obtained from Luciano's thesis.

In [ ]:
# Fuel rod surfaces:

fuel_outer_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = UO2_expansion(0.4096, 298, Tf))
gap_outer_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = zirconium_expansion(0.4096 + 0.0083, 298, T))
clad_outer_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = zirconium_expansion(0.4096 + 0.0083 + 0.0572, 298, T))

# Fuel rod regions:

fuel_region = -fuel_outer_radius
gap_region = -gap_outer_radius & +fuel_outer_radius
clad_region = -clad_outer_radius & +gap_outer_radius
moderator_region = +clad_outer_radius

# Fuel rod cells:

fuel_cell_R1 = openmc.Cell(fill = UO2_21, region = fuel_region)
fuel_cell_R2 = openmc.Cell(fill = UO2_26, region = fuel_region)
fuel_cell_R3 = openmc.Cell(fill = UO2_31, region = fuel_region)
gap_cell_R1 = openmc.Cell(fill = Helium, region = gap_region)
gap_cell_R2 = openmc.Cell(fill = Helium, region = gap_region)
gap_cell_R3 = openmc.Cell(fill = Helium, region = gap_region)
clad_cell_R1 = openmc.Cell(fill = Zircaloy, region = clad_region)
clad_cell_R2 = openmc.Cell(fill = Zircaloy, region = clad_region)
clad_cell_R3 = openmc.Cell(fill = Zircaloy, region = clad_region)
moderator_cell_R1 = openmc.Cell(fill = H2O, region = moderator_region)
moderator_cell_R2 = openmc.Cell(fill = H2O, region = moderator_region)
moderator_cell_R3 = openmc.Cell(fill = H2O, region = moderator_region)

# Fuel rod universes:

F1 = openmc.Universe(cells = [fuel_cell_R1, gap_cell_R1, clad_cell_R1, moderator_cell_R1])
F2 = openmc.Universe(cells = [fuel_cell_R2, gap_cell_R2, clad_cell_R2, moderator_cell_R2])
F3 = openmc.Universe(cells = [fuel_cell_R3, gap_cell_R3, clad_cell_R3, moderator_cell_R3])

### Control rod geometry

Control rod radii obtained from Luciano's thesis.

In [ ]:
radii_list = [0.417830, 0.421640, 0.466090, 0.552450, 0.598170]
materials_list = [AgInCd, Air, Steel, H2O, Zircaloy, H2O]

innermost_cell = openmc.Cell(fill = materials_list[0], region = -openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[0]))
outermost_cell = openmc.Cell(fill = materials_list[-1], region = +openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[-1]))

cells_list = [innermost_cell]

for i in range(len(radii_list) - 1):
    inner_surface = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[i])
    outer_surface = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[i + 1])
    region = +inner_surface & -outer_surface
    cell = openmc.Cell(fill = materials_list[i + 1], region = region)
    cells_list.append(cell)

cells_list.append(outermost_cell)

CR = openmc.Universe(cells = cells_list)

### Burnable poison rod geometry

Burnable poison rod radii obtained from Luciano's thesis.

In [ ]:
radii_list = [0.198755, 0.215265, 0.226060, 0.411480, 0.421640, 0.466090, 0.552450, 0.598170]
materials_list = [Air, Steel, Air, Borosilicate, Air, Steel, H2O, Zircaloy, H2O]

innermost_cell = openmc.Cell(fill = materials_list[0], region = -openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[0]))
outermost_cell = openmc.Cell(fill = materials_list[-1], region = +openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[-1]))

cells_list = [innermost_cell]

for i in range(len(radii_list) - 1):
    inner_surface = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[i])
    outer_surface = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[i + 1])
    region = +inner_surface & -outer_surface
    cell = openmc.Cell(fill = materials_list[i + 1], region = region)
    cells_list.append(cell)

cells_list.append(outermost_cell)

BR = openmc.Universe(cells = cells_list)

### Empty rod geometry

Empty rod radii obtained from Luciano's thesis.

In [ ]:
radii_list = [0.552450, 0.598170]
materials_list = [H2O, Zircaloy, H2O]

innermost_cell = openmc.Cell(fill = materials_list[0], region = -openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[0]))
outermost_cell = openmc.Cell(fill = materials_list[-1], region = +openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[-1]))

cells_list = [innermost_cell]

for i in range(len(radii_list) - 1):
    inner_surface = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[i])
    outer_surface = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = radii_list[i + 1])
    region = +inner_surface & -outer_surface
    cell = openmc.Cell(fill = materials_list[i + 1], region = region)
    cells_list.append(cell)

cells_list.append(outermost_cell)

ER = openmc.Universe(cells = cells_list)

### Fuel assembly geometry

Fuel assembly types obtained from Luciano's thesis.

In [ ]:
# Pitch of the fuel assembly:

pitch = 1.233598

# Size of the fuel assembly lattice:

N = 16

# Fuel assembly lattices:

E21F1 = [[F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, F1, F1, F1],
         [F1, F1, F1, ER, F1, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, ER, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, ER, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, ER, F1, F1, F1, ER, F1, F1, F1],
         [F1, F1, F1, ER, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, ER, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, ER, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, F1, ER, F1, F1, F1],
         [F1, F1, F1, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1]]

E21F2 = [[F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, ER, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, ER, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, ER, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, ER, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2]]

E21F3 = [[F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, F3, F3, F3],
         [F3, F3, F3, ER, F3, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, ER, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, ER, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, ER, F3, F3, F3, ER, F3, F3, F3],
         [F3, F3, F3, ER, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, ER, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, ER, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, F3, ER, F3, F3, F3],
         [F3, F3, F3, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3]]

C20F1 = [[F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, CR, F1, F1, F1, F1, CR, F1, F1, F1, F1, F1],
         [F1, F1, F1, CR, F1, F1, F1, CR, F1, F1, F1, F1, CR, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, CR, F1, F1, CR, F1, F1, F1, F1, CR, F1, F1, CR, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, ER, F1, F1, F1, CR, F1, F1, F1],
         [F1, F1, F1, CR, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, CR, F1, F1, CR, F1, F1, F1, F1, CR, F1, F1, CR, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, CR, F1, F1, F1, F1, CR, F1, F1, F1, CR, F1, F1, F1],
         [F1, F1, F1, F1, F1, CR, F1, F1, F1, F1, CR, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1]]

C20F2 = [[F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, CR, F2, F2, F2, F2, CR, F2, F2, F2, F2, F2],
         [F2, F2, F2, CR, F2, F2, F2, CR, F2, F2, F2, F2, CR, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, CR, F2, F2, CR, F2, F2, F2, F2, CR, F2, F2, CR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, ER, F2, F2, F2, CR, F2, F2, F2],
         [F2, F2, F2, CR, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, CR, F2, F2, CR, F2, F2, F2, F2, CR, F2, F2, CR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, CR, F2, F2, F2, F2, CR, F2, F2, F2, CR, F2, F2, F2],
         [F2, F2, F2, F2, F2, CR, F2, F2, F2, F2, CR, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2]]

B08F1 = [[F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, BR, F1, F1, F1, F1, BR, F1, F1, F1, F1, F1],
         [F1, F1, F1, ER, F1, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, BR, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, BR, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, ER, F1, F1, F1, ER, F1, F1, F1],
         [F1, F1, F1, ER, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, BR, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, BR, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, ER, F1, F1, F1, F1, ER, F1, F1, F1, ER, F1, F1, F1],
         [F1, F1, F1, F1, F1, BR, F1, F1, F1, F1, BR, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1],
         [F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1, F1]]

B08F2 = [[F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, BR, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, BR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, BR, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, BR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2]]

B08F3 = [[F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, BR, F3, F3, F3, F3, BR, F3, F3, F3, F3, F3],
         [F3, F3, F3, ER, F3, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, BR, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, BR, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, ER, F3, F3, F3, ER, F3, F3, F3],
         [F3, F3, F3, ER, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, BR, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, BR, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, ER, F3, F3, F3, F3, ER, F3, F3, F3, ER, F3, F3, F3],
         [F3, F3, F3, F3, F3, BR, F3, F3, F3, F3, BR, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3],
         [F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3, F3]]

B12F2 = [[F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, BR, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, BR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, BR, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, BR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2]]

B16F2 = [[F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, F2, F2, F2],
         [F2, F2, F2, BR, F2, F2, F2, ER, F2, F2, F2, F2, BR, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, BR, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, BR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, ER, F2, F2, F2, ER, F2, F2, F2],
         [F2, F2, F2, ER, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, BR, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, BR, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, BR, F2, F2, F2, F2, ER, F2, F2, F2, BR, F2, F2, F2],
         [F2, F2, F2, F2, F2, BR, F2, F2, F2, F2, BR, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2],
         [F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2, F2]]

### Active core geometry

Core map obtained from Westinghouses's Angra 1 Cycle 1 Data Book.

In [ ]:
# Size of active core lattice:

NA = 13

# Active core map:

core_map = [['OUT', 'OUT', 'OUT', 'OUT', 'OUT', E21F3, E21F3, E21F3, 'OUT', 'OUT', 'OUT', 'OUT', 'OUT'],
            ['OUT', 'OUT', 'OUT', E21F3, E21F3, E21F2, B08F2, E21F2, E21F3, E21F3, 'OUT', 'OUT', 'OUT'],
            ['OUT', 'OUT', E21F3, B08F3, E21F1, B12F2, E21F1, B12F2, E21F1, B08F3, E21F3, 'OUT', 'OUT'],
            ['OUT', E21F3, B08F3, E21F1, B12F2, E21F1, B16F2, E21F1, B12F2, E21F1, B08F3, E21F3, 'OUT'],
            ['OUT', E21F3, E21F1, B12F2, E21F1, B16F2, E21F1, B16F2, E21F1, B12F2, E21F1, E21F3, 'OUT'],
            [E21F3, E21F2, B12F2, E21F1, B16F2, E21F1, B08F1, E21F1, B16F2, E21F1, B12F2, E21F2, E21F3],
            [E21F3, B08F2, E21F1, B16F2, E21F1, B08F1, E21F1, B08F1, E21F1, B16F2, E21F1, B08F2, E21F3],
            [E21F3, E21F2, B12F2, E21F1, B16F2, E21F1, B08F1, E21F1, B16F2, E21F1, B12F2, E21F2, E21F3],
            ['OUT', E21F3, E21F1, B12F2, E21F1, B16F2, E21F1, B16F2, E21F1, B12F2, E21F1, E21F3, 'OUT'],
            ['OUT', E21F3, B08F3, E21F1, B12F2, E21F1, B16F2, E21F1, B12F2, E21F1, B08F3, E21F3, 'OUT'],
            ['OUT', 'OUT', E21F3, B08F3, E21F1, B12F2, E21F1, B12F2, E21F1, B08F3, E21F3, 'OUT', 'OUT'],
            ['OUT', 'OUT', 'OUT', E21F3, E21F3, E21F2, B08F2, E21F2, E21F3, E21F3, 'OUT', 'OUT', 'OUT'],
            ['OUT', 'OUT', 'OUT', 'OUT', 'OUT', E21F3, E21F3, E21F3, 'OUT', 'OUT', 'OUT', 'OUT', 'OUT']]

# X planes list:

x_planes = []
for i in range(NA + 1):
    x_planes.append(openmc.XPlane(x0 = -(pitch*N*NA/2) + (pitch*N*i)))
    
# Y planes list:

y_planes = []
for j in range(NA + 1):
    y_planes.append(openmc.YPlane(y0 = -(pitch*N*NA/2) + (pitch*N*j)))

# Active core region and cells:

regions_list = []
fuel_assembly_list = []

for i in range(NA):
    for j in range(NA):
        if core_map[i][j] != 'OUT':
            region = +x_planes[i] & -x_planes[i + 1] & +y_planes[j] & -y_planes[j + 1]
            regions_list.append(region)
            lattice = openmc.RectLattice()
            lattice.pitch = (pitch, pitch)
            lattice.lower_left = [x_planes[i].x0, y_planes[j].y0]
            lattice.universes = core_map[i][j]
            cell = openmc.Cell(fill = lattice, region = region)
            fuel_assembly_list.append(cell)
            
active_core_region = openmc.Union(regions_list)

Reflector thickness obtained from Luciano's thesis.

In [ ]:
reflector_thickness = 2.86

reflector_x_planes = []
reflector_y_planes = []

for plane in x_planes:
    if plane.x0 > 0:
        reflector_plane = openmc.XPlane(x0 = plane.x0 + reflector_thickness)
        reflector_x_planes.append(reflector_plane)
    if plane.x0 < 0:
        reflector_plane = openmc.XPlane(x0 = plane.x0 - reflector_thickness)
        reflector_x_planes.append(reflector_plane)
        
for plane in y_planes:
    if plane.y0 > 0:
        reflector_plane = openmc.YPlane(y0 = plane.y0 + reflector_thickness)
        reflector_y_planes.append(reflector_plane)
    if plane.y0 < 0:
        reflector_plane = openmc.YPlane(y0 = plane.y0 - reflector_thickness)
        reflector_y_planes.append(reflector_plane)

reflector_regions_list = []

for i in range(NA):
    for j in range(NA):
        if core_map[i][j] != 'OUT':
            region = +reflector_x_planes[i] & -reflector_x_planes[i + 1] & +reflector_y_planes[j] & -reflector_y_planes[j + 1]
            reflector_regions_list.append(region)
            
reflector_region = openmc.Union(reflector_regions_list) & ~active_core_region

reflector = openmc.Cell(fill = Steel, region = reflector_region)

### Exterior of the active core

Core barrel, thermal shield and reactor vessel radii and thicknesses obtained from Luciano's thesis.

In [ ]:
# Exterior of the active core surfaces:

core_barrel_inner_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = 138.43)
core_barrel_outer_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = 138.43 + 4.53)

thermal_shield_inner_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = 146.40)
thermal_shield_outer_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = 146.40 + 9.10)

reactor_vessel_inner_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = 167.64)
reactor_vessel_outer_radius = openmc.ZCylinder(x0 = 0.0, y0 = 0.0, r = 167.64 + 16.83, boundary_type = 'vacuum')

# Exterior of the active core regions:

water_region_I = ~openmc.Union([active_core_region, reflector_region]) & -core_barrel_inner_radius

core_barrel_region = +core_barrel_inner_radius & -core_barrel_outer_radius

water_region_II = +core_barrel_outer_radius & -thermal_shield_inner_radius

thermal_shield_region = +thermal_shield_inner_radius & -thermal_shield_outer_radius

water_region_III = +thermal_shield_outer_radius & -reactor_vessel_inner_radius

reactor_vessel_region = +reactor_vessel_inner_radius & -reactor_vessel_outer_radius

# Exterior of active core cells:

water_cell_I = openmc.Cell(fill = H2O, region = water_region_I)

core_barrel = openmc.Cell(fill = Steel, region = core_barrel_region)

water_cell_II = openmc.Cell(fill = H2O, region = water_region_II)

thermal_shield = openmc.Cell(fill = Inconel, region = thermal_shield_region)

water_cell_III = openmc.Cell(fill = H2O, region = water_region_III)

reactor_vessel = openmc.Cell(fill = Inconel, region = reactor_vessel_region)

### Packing up the geometry

In [ ]:
# Height of the model:

height = 1.00

# Surfaces that bound the model and their boundary conditions:

min_z = openmc.ZPlane(z0 = -height/2, boundary_type = 'periodic')
max_z = openmc.ZPlane(z0 = +height/2, boundary_type = 'periodic')

min_z.periodic_surface = max_z

# Full core universe:

full_core = openmc.Universe(name = 'Full Core', cells = fuel_assembly_list + [reflector, water_cell_I, core_barrel, water_cell_II, thermal_shield, water_cell_III, reactor_vessel])

# Root cell:

root_cell = openmc.Cell(name = 'Root Cell', fill = full_core, region = +min_z & -max_z)

# Root universe:

root_universe = openmc.Universe(name = 'Root Universe', cells = [root_cell])

# Exporting the geometry to XML:

geometry = openmc.Geometry(root = root_universe)
geometry.export_to_xml()

# Tallies

In [ ]:
# Universe Tally:

universe_filter = openmc.UniverseFilter(root_universe)

T1 = openmc.Tally(1, name = 'Universe Tally')
T1.filters = [universe_filter]
T1.scores = ['heating-local', 'prompt-nu-fission', 'delayed-nu-fission']

# Energy Tally:

energy_bins = np.logspace(start = -3, stop = 7, num = 1000, endpoint = True, base = 10)
energy_filter = openmc.EnergyFilter(energy_bins)

T2 = openmc.Tally(2, name = 'Energy Tally')
T2.filters = [energy_filter]
T2.scores = ['flux']

# Assembly Mesh Tally:

assembly_mesh = openmc.RegularMesh()
assembly_mesh.dimension = [NA, NA, 1]
assembly_mesh.lower_left = [-pitch*N*NA/2, -pitch*N*NA/2, -height/2]
assembly_mesh.upper_right = [pitch*N*NA/2, pitch*N*NA/2, height/2]

assembly_mesh_filter = openmc.MeshFilter(assembly_mesh)

T3 = openmc.Tally(3, name = 'Assembly Mesh Tally')
T3.filters = [assembly_mesh_filter]
T3.scores = ['flux', 'heating-local']

# Rod Mesh Tally:

rod_mesh = openmc.RegularMesh()
rod_mesh.dimension = [N*NA, N*NA, 1]
rod_mesh.lower_left = [-pitch*N*NA/2, -pitch*N*NA/2, -height/2]
rod_mesh.upper_right = [pitch*N*NA/2, pitch*N*NA/2, height/2]

rod_mesh_filter = openmc.MeshFilter(rod_mesh)

T4 = openmc.Tally(4, name = 'Rod Mesh Tally')
T4.filters = [rod_mesh_filter]
T4.scores = ['flux', 'heating-local']

# Exporting the tallies list to XML:

tallies = openmc.Tallies([T1, T2, T3, T4])
tallies.export_to_xml()

# Settings

In [ ]:
# Defining a settings object:

settings = openmc.Settings()

## Initial particle source

In [ ]:
# Spatial distribution that includes all regions with fissile material:

spatial_dist = openmc.stats.Box([-pitch*N*NA/2, -pitch*N*NA/2, -height/2], [pitch*N*NA/2, pitch*N*NA/2, height/2], only_fissionable = True)

# Isotropic angular distribution:

angular_dist = openmc.stats.Isotropic()

# Watt spectrum energy distribution:

energy_dist = openmc.stats.Watt()

# Defining the initial particle source:

origin = openmc.Source(space = spatial_dist, angle = angular_dist, energy = energy_dist, particle = 'neutron')

# Setting the source variable:

settings.source = origin

## Shannon entropy mesh

In [ ]:
entropy_mesh = openmc.RegularMesh()
entropy_mesh.lower_left = [-pitch*N*NA/2, -pitch*N*NA/2, -height/2]
entropy_mesh.upper_right = [pitch*N*NA/2, pitch*N*NA/2, height/2]
entropy_mesh.dimension = [4*NA, 4*NA, 1]
settings.entropy_mesh = entropy_mesh

## Simulation parameters

In [ ]:
# Execution mode:

settings.run_mode = 'eigenvalue'
settings.photon_transport = False
settings.create_fission_neutrons = True
settings.energy_mode = 'continuous-energy'
settings.electron_treatment = 'led'

# Number of particles and batches:

settings.particles = 54080
settings.generations_per_batch = 3
settings.inactive = 50
settings.batches = 250

# Temperature treatment:

settings.temperature = {'default': T, 'method': 'interpolation', 'multipole': False}

# Uncertainty reporting mode:

settings.confidence_intervals = False

# Output files:

settings.output = {'path': './', 'summary': True, 'tallies': False}

# Exporting settings to XML:

settings.export_to_xml()

# Geometry ploting mode

## Full core plot

In [ ]:
TR = 167.64 + 16.83
full_core_plot = openmc.Plot(name = 'Angra 1 Full Core Plot')
full_core_plot.basis = 'xy'
full_core_plot.origin = (0.0, 0.0, 0.0)
full_core_plot.width = (2*TR, 2*TR)
full_core_plot.pixels = (15000, 15000)
full_core_plot.color_by = 'material'
full_core_plot.colors = {UO2_21: 'yellow', UO2_26: 'pink', UO2_31: 'orange', Helium: 'red', Zircaloy: 'darkgray', 
                         H2O: 'blue', AgInCd: 'goldenrod', Borosilicate: 'goldenrod', Steel: 'darkgray', Inconel: 'darkgray', Air: 'azure'}
full_core_plot.show_overlaps = True
full_core_plot.overlap_color = 'black'

## Assembly plot

In [ ]:
assembly_plot = openmc.Plot(name = 'Angra 1 Assembly Plot')
assembly_plot.basis = 'xy'
assembly_plot.origin = (2*N*pitch, 3*N*pitch, 0.0)
assembly_plot.width = (N*pitch, N*pitch)
assembly_plot.pixels = (2000, 2000)
assembly_plot.color_by = 'material'
assembly_plot.colors = {UO2_21: 'yellow', UO2_26: 'pink', UO2_31: 'orange', Helium: 'red', Zircaloy: 'darkgray', 
                         H2O: 'blue', AgInCd: 'goldenrod', Borosilicate: 'goldenrod', Steel: 'darkgray', Inconel: 'darkgray', Air: 'azure'}
assembly_plot.show_overlaps = True
assembly_plot.overlap_color = 'black'

## Generating the plots

In [ ]:
geometry_plots = openmc.Plots([full_core_plot])
geometry_plots.export_to_xml()
openmc.plot_geometry()

# Running the simulation

In [ ]:
openmc.run()